<a href="https://colab.research.google.com/drive/11jE0hlhnp-ryKqu7LQARvMAQOeK2qdj1?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### Decomposed Prompting

In [1]:
!pip install -qU google-generativeai

In [2]:
import google.generativeai as genai
import getpass

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [7]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [9]:
genai.configure(api_key=API_KEY)

In [10]:
class SymbolicMemory:
    """Tracks intermediate results and context"""
    def __init__(self):
        self.memory = {}

    def store(self, key, value):
        self.memory[key] = value

    def retrieve(self, key):
        return self.memory.get(key)

    def get_all(self):
        return self.memory.copy()

    def __str__(self):
        return "\n".join([f"{k}: {v}" for k, v in self.memory.items()])

class DecomposedPromptingAgent:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.0-flash-exp")
        self.memory = SymbolicMemory()

    def identify_subtasks(self, task):
        """Step 1: Identify sub-tasks"""
        prompt = f"""Break down this complex task into 3-6 smaller sub-tasks.
        Each sub-task should be specific and focused.

        Task: {task}

        List sub-tasks numbered:"""

        response = self.model.generate_content(prompt).text

        # Parse sub-tasks
        subtasks = []
        for line in response.split("\n"):
            line = line.strip()
            if line and (line[0].isdigit() or line.startswith("-")):
                subtask = line.lstrip("0123456789.-) ").strip()
                if subtask and len(subtask) > 10:
                    subtasks.append(subtask)

        return subtasks

    def create_specialized_prompt(self, subtask, task_context):
        """Step 2: Create tailored prompt for sub-task"""
        memory_context = str(self.memory) if self.memory.get_all() else "None yet"

        prompt = f"""Original Task: {task_context}

        Current Sub-Task: {subtask}

        Previously Computed Results:
        {memory_context}

        Solve this sub-task. Be specific and show your work:"""

        return prompt

    def solve_subtask(self, subtask, task_context, subtask_id):
        """Step 3: Solve individual sub-task"""
        specialized_prompt = self.create_specialized_prompt(subtask, task_context)

        response = self.model.generate_content(specialized_prompt).text
        result = response.strip()

        # Store in symbolic memory
        self.memory.store(f"subtask_{subtask_id}", result)

        return result

    def integrate_results(self, task, subtasks, results):
        """Step 4: Coordinate and integrate all sub-task results"""
        subtask_summary = "\n\n".join([
            f"Sub-task {i+1}: {subtask}\nResult: {result}"
            for i, (subtask, result) in enumerate(zip(subtasks, results))
        ])

        prompt = f"""Original Task: {task}

        Sub-task Results:
        {subtask_summary}

        Integrate these results into a comprehensive final answer:"""

        response = self.model.generate_content(prompt).text
        return response.strip()

    def solve(self, task):
        """Main decomposed prompting pipeline"""
        print(f"\n{'='*60}")
        print(f"🧩 Decomposed Prompting")
        print(f"{'='*60}")
        print(f"Task: {task}\n")

        # Reset memory
        self.memory = SymbolicMemory()

        # Step 1: Identify sub-tasks
        print(f"{'─'*60}")
        print(f"STEP 1: Identifying Sub-Tasks")
        print(f"{'─'*60}\n")

        subtasks = self.identify_subtasks(task)

        print(f"Identified {len(subtasks)} sub-tasks:\n")
        for i, subtask in enumerate(subtasks, 1):
            print(f"{i}. {subtask}")
        print()

        # Step 2 & 3: Create specialized prompts and solve
        print(f"{'─'*60}")
        print(f"STEP 2-3: Solving Sub-Tasks with Specialized Prompts")
        print(f"{'─'*60}\n")

        results = []

        for i, subtask in enumerate(subtasks, 1):
            print(f"📍 Sub-task {i}/{len(subtasks)}: {subtask[:60]}...")

            result = self.solve_subtask(subtask, task, i)
            results.append(result)

            print(f"   ✓ Result: {result[:100]}...")
            print(f"   💾 Stored in memory as 'subtask_{i}'\n")

        # Show symbolic memory
        print(f"{'─'*60}")
        print(f"📚 Symbolic Memory State")
        print(f"{'─'*60}")
        print(self.memory)
        print()

        # Step 4: Integrate results
        print(f"{'─'*60}")
        print(f"STEP 4: Integrating Results")
        print(f"{'─'*60}\n")

        final_answer = self.integrate_results(task, subtasks, results)

        print(f"{'='*60}")
        print(f"💡 FINAL ANSWER")
        print(f"{'='*60}")
        print(final_answer)
        print()

        return final_answer

In [11]:
# Example 1: Complex Math Problem
print("="*60)
print("EXAMPLE 1: Complex Math Problem")
print("="*60)

agent1 = DecomposedPromptingAgent()
agent1.solve(
    "A company's revenue was $500k in Year 1. It grew 20% in Year 2, then decreased 10% in Year 3. "
    "In Year 4, it grew by the average of Year 2 and Year 3 growth rates. What was the Year 4 revenue? "
    "Also calculate the total revenue over all 4 years."
)


# Example 2: Multi-Step Reasoning
print("\n" + "="*60)
print("EXAMPLE 2: Multi-Step Logic Problem")
print("="*60)

agent2 = DecomposedPromptingAgent()
agent2.solve(
    "Five people - Alice, Bob, Carol, Dave, and Emma - are standing in a line. "
    "Alice is not at either end. Bob is between Carol and Dave. Emma is at one end. "
    "Dave is not next to Emma. What is the order from left to right?"
)


# Example 3: Algorithm Design
print("\n" + "="*60)
print("EXAMPLE 3: Algorithm Design")
print("="*60)

agent3 = DecomposedPromptingAgent()
agent3.solve(
    "Design an algorithm to find the second largest element in an unsorted array. "
    "Explain the approach, write pseudocode, analyze time complexity, and identify edge cases."
)


# Example 4: Scientific Problem
print("\n" + "="*60)
print("EXAMPLE 4: Scientific Analysis")
print("="*60)

agent4 = DecomposedPromptingAgent()
agent4.solve(
    "A ball is thrown upward with initial velocity 20 m/s from a height of 5 meters. "
    "Calculate: (1) maximum height reached, (2) time to reach max height, "
    "(3) total time in air, (4) velocity when hitting ground. Use g = 10 m/s²."
)


# Example 5: Project Planning
print("\n" + "="*60)
print("EXAMPLE 5: Project Planning")
print("="*60)

agent5 = DecomposedPromptingAgent()
agent5.solve(
    "Plan a company website redesign project with $50k budget and 3-month timeline. "
    "Identify phases, allocate budget, assign timeline, list deliverables, and identify risks."
)


# Example 6: Code Debugging Workflow
print("\n" + "="*60)
print("EXAMPLE 6: Debugging Workflow")
print("="*60)

agent6 = DecomposedPromptingAgent()
agent6.solve(
    "A Python function to calculate factorial is returning incorrect results for large numbers. "
    "Diagnose potential issues: check logic, identify edge cases, consider data types, "
    "suggest fixes, and recommend testing approach."
)


# Example 7: Business Analysis
print("\n" + "="*60)
print("EXAMPLE 7: Business Analysis")
print("="*60)

agent7 = DecomposedPromptingAgent()
agent7.solve(
    "Analyze whether to expand a coffee shop chain into a new city. "
    "Research market size, estimate costs, project revenue, calculate ROI, "
    "identify risks, and make a recommendation."
)


# Example 8: Educational Problem
print("\n" + "="*60)
print("EXAMPLE 8: Educational Exercise")
print("="*60)

agent8 = DecomposedPromptingAgent()
agent8.solve(
    "Solve this chemistry problem: How many grams of water are produced when "
    "10g of hydrogen reacts completely with oxygen? "
    "Show: balanced equation, molar calculations, stoichiometry, and final answer."
)


print("✅ Decomposed Prompting Complete!")

EXAMPLE 1: Complex Math Problem

🧩 Decomposed Prompting
Task: A company's revenue was $500k in Year 1. It grew 20% in Year 2, then decreased 10% in Year 3. In Year 4, it grew by the average of Year 2 and Year 3 growth rates. What was the Year 4 revenue? Also calculate the total revenue over all 4 years.

────────────────────────────────────────────────────────────
STEP 1: Identifying Sub-Tasks
────────────────────────────────────────────────────────────

Identified 5 sub-tasks:

1. **Calculate Year 2 Revenue:** Determine the revenue for Year 2 by applying the 20% growth to the Year 1 revenue.
2. **Calculate Year 3 Revenue:** Determine the revenue for Year 3 by applying the 10% decrease to the Year 2 revenue.
3. **Calculate Average Growth Rate:** Calculate the average of the Year 2 growth rate (20%) and the Year 3 growth rate (-10%).
4. **Calculate Year 4 Revenue:** Determine the revenue for Year 4 by applying the average growth rate (calculated in the previous step) to the Year 3 reven

TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-exp:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota. Please migrate to Gemini 2.0 Flash Preview (Image Generation) (models/gemini-2.0-flash-preview-image-generation) for higher quota limits. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.